# Additional information: Linear (OLS) Regression

This extra notebook shows how you can do hypothesis testing with Linear (OLS)regression using statsmodels. 

Here we will be using a dataset from scikit learn about [Boston house prices](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html). We will try to the price (MEDV) on the basis of age, and crime in the region. (Note: Scikit-learn datasets are not allowed for A1; variables below are standardized)

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from sklearn.datasets import load_diabetes

In [3]:
from sklearn.datasets import load_boston
data = load_boston()

In [4]:
MEDV = data['target']

# 1. Data cleaning
(Note: same steps as in the tutorial)

In [5]:
features = pd.DataFrame(data['data'], columns = data['feature_names'])
target = pd.DataFrame(MEDV, columns = ['MEDV',])
features = features.reset_index()
target = target.reset_index()
data = features.merge(target)
# Note: I'm using some workarounds to get the features and the target 
# in the same dataframe, as sklearn datasets come usually in other formats

In [6]:
data.head()

,index,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0.006,18.000,2.310,0.000,0.538,6.575,65.200,4.090,1.000,296.000,15.300,396.900,4.980,24.000
1,1,0.027,0.000,7.070,0.000,0.469,6.421,78.900,4.967,2.000,242.000,17.800,396.900,9.140,21.600
2,2,0.027,0.000,7.070,0.000,0.469,7.185,61.100,4.967,2.000,242.000,17.800,392.830,4.030,34.700
3,3,0.032,0.000,2.180,0.000,0.458,6.998,45.800,6.062,3.000,222.000,18.700,394.630,2.940,33.400
4,4,0.069,0.000,2.180,0.000,0.458,7.147,54.200,6.062,3.000,222.000,18.700,396.900,5.330,36.200


In [7]:
data.describe()

,index,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000,506.000
mean,252.500,3.614,11.364,11.137,0.069,0.555,6.285,68.575,3.795,9.549,408.237,18.456,356.674,12.653,22.533
std,146.214,8.602,23.322,6.860,0.254,0.116,0.703,28.149,2.106,8.707,168.537,2.165,91.295,7.141,9.197
min,0.000,0.006,0.000,0.460,0.000,0.385,3.561,2.900,1.130,1.000,187.000,12.600,0.320,1.730,5.000
25%,126.250,0.082,0.000,5.190,0.000,0.449,5.886,45.025,2.100,4.000,279.000,17.400,375.377,6.950,17.025
50%,252.500,0.257,0.000,9.690,0.000,0.538,6.208,77.500,3.207,5.000,330.000,19.050,391.440,11.360,21.200
75%,378.750,3.677,12.500,18.100,0.000,0.624,6.623,94.075,5.188,24.000,666.000,20.200,396.225,16.955,25.000
max,505.000,88.976,100.000,27.740,1.000,0.871,8.780,100.000,12.127,24.000,711.000,22.000,396.900,37.970,50.000


## 2. Hypothesis testing with statsmodels


The main change here is that we use OLS, instead of logit

In [8]:
import statsmodels.api as sm


In [9]:
import statsmodels.formula.api as smf

In [10]:
features = ['AGE', 'CRIM']

In [11]:
OLS_model = sm.OLS(data['MEDV'], sm.add_constant(data[features]))




In [12]:
result = OLS_model.fit()

In [13]:
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     69.52
Date:                Tue, 16 Mar 2021   Prob (F-statistic):           2.20e-27
Time:                        11:22:43   Log-Likelihood:                -1778.5
No. Observations:                 506   AIC:                             3563.
Df Residuals:                     503   BIC:                             3576.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         29.8007      0.971     30.698      0.0

# Using scikit learn to predict cases

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
clf_OLS = LinearRegression()

In [16]:
clf_OLS.fit(data[features], data['MEDV'])

LinearRegression()

In [17]:
pd.DataFrame(np.transpose(clf_OLS.coef_), features)

,0
AGE,-0.090
CRIM,-0.312


Testing the median price for two cases with age varying (10 or 50) keeping criminality constant

In [18]:
cases = [[[10, 5]], [[50,5]]]

In [19]:
for case in cases:
    print(case, ':', clf_OLS.predict(case))

[[10, 5]] : [27.34605542]
[[50, 5]] : [23.76392437]


Testing two cases with age constant but criminality varying (min and max)

In [20]:
cases = [[[10, 0.006]], [[10,88.976]]]

In [21]:
for case in cases:
    print(case, ':', clf_OLS.predict(case))

[[10, 0.006]] : [28.90326335]
[[10, 88.976]] : [1.16101467]


# 3. Adding interactions (to statsmodels)

We will use statsmodels, but with formulas similar to R. 

To have a dependent variable (DV) predicted by the interaction of two independent variables (IV1 and IV2), we use:

```DV ~ IV1 * IV2``` 

Please note that we are using a different module of statsmodels (smf, not sm), which we import with:

```import statsmodels.formula.api as smf```


In [22]:
formula = 'MEDV ~ AGE * CRIM '

In [23]:
OLS_model_int = smf.ols(formula, data=data)

In [24]:
result_interaction = OLS_model_int.fit()

In [25]:
print(result_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     47.43
Date:                Tue, 16 Mar 2021   Prob (F-statistic):           5.33e-27
Time:                        11:22:43   Log-Likelihood:                -1777.1
No. Observations:                 506   AIC:                             3562.
Df Residuals:                     502   BIC:                             3579.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.0866      0.984     30.568      0.0